Crawler para extração dos itens do carrinho da Atividade Prática.

# Imports

In [21]:
import requests, json, threading, sys, traceback
import pandas as pd

# Funções

In [22]:
def get_category(category: str) -> list:
    headers = {
        "Content-Type": "application/json",
        "Accept": "application/json",
    }

    payload = {
        "partner": "linx",
        "page": 1,
        "resultsPerPage": 300,
        "multiCategory": category,
        "sortBy": "relevance",
        "department": "ecom",
        "storeId": 461,
        "customerPlus": True
    }
    
    try:
        # Pega o número total de produtos da categoria
        response = requests.post(r"https://api.vendas.gpa.digital/pa/search/category-page", headers=headers, data=json.dumps(payload))
        response.raise_for_status()
        max_produtos = response.json()['totalProducts']
        payload['resultsPerPage'] = max_produtos

        # Pega a lista de produtos da categoria
        response = requests.post(r"https://api.vendas.gpa.digital/pa/search/category-page", headers=headers, data=json.dumps(payload))
        response.raise_for_status()
        
        lista_p = list(response.json()['products'])
        lista_cod = []

        for prod in lista_p:
            lista_cod.append(prod['id'])
            
    except:
        return []
    
    else:
        return lista_cod

In [23]:
def get_product(code: int, verbose: int = 0) -> dict:
    try:
        # Fazendo a requisição
        texto = requests.get(fr"https://www.paodeacucar.com/produto/{code}").text
        
        # Encontrando as posições
        inicio = texto.find('"props":') + len('"props":')
        fim = texto.find(',"page":"/produto"', inicio)

        # Extraindo a substring
        substring = texto[inicio:fim].strip()
        
        # Dicionario Geral
        dicionario = json.loads(substring)
        
        # Informações gerais do produto
        informacoes = dicionario['initialProps']['componentProps']['product']
        atributos = get_dictkey(informacoes, 'attributeGroups')
        
        # Dicionário com os resultados
        resultados = {    
        'id': informacoes['id'], 
        'nome': informacoes['name'], 
        'sku': informacoes['sku'], 
        'descricao': informacoes['description'],
        'imagem': f'https://static.paodeacucar.com{informacoes['thumbPath']}',
        'preco': get_dictkey(informacoes, 'currentPrice'),
        'marca': get_dictcode(atributos, 'Marca'),
        'alturaProduto': get_dictcode(atributos, 'alturaProduto'),
        'larguraProduto': get_dictcode(atributos, 'larguraProduto'),
        'pesoBruto': get_dictcode(atributos, 'pesoBruto'),
        'unidadePesoBruto': get_dictcode(atributos, 'unidadePesoBruto'),
        'profundidadeProduto': get_dictcode(atributos, 'profundidadeProduto')
        }
        
        assert(
            resultados['id'] == code and 
            resultados['preco'] != None and
            resultados['alturaProduto'] != None and 
            resultados['larguraProduto'] != None and 
            resultados['pesoBruto'] != None and 
            resultados['unidadePesoBruto'] != None and
            resultados['profundidadeProduto'] != None
        )

    except:
        if verbose >= 1:
            exc_info = sys.exc_info()
            e = ''.join(traceback.format_exception(*exc_info))
            del exc_info
            
            print(f"Erro ao extrair informações do produto {code}!\nErro:\n{e}")
        
        if verbose >= 2:     
            if 'informacoes' in locals():
                print(json.dumps(locals()['informacoes'], indent=2))
                
            if 'informacoes2' in locals():
                print(json.dumps(locals()['informacoes2'], indent=2))
        
        return None
    
    else:
        return resultados

In [24]:
def fetch_products(codigos, verbose: int = 0):
    resultados = []
    threads = []
    lock = threading.Lock()

    def thread_function(codigo, verbose):
        info = get_product(codigo, verbose)
        
        if info:
            with lock:
                resultados.append(info)

    for codigo in codigos:
        thread = threading.Thread(target=thread_function, args=(codigo, verbose,))
        threads.append(thread)
        thread.start()

    while any(thread.is_alive() for thread in threads): pass

    return resultados


In [25]:
def get_dictkey(dicionario, chave):
    # Função interna recursiva
    def busca_recursiva(d):
        if isinstance(d, dict):
            for k, v in d.items():
                if k == chave:
                    return v
                resultado = busca_recursiva(v)
                if resultado is not None:
                    return resultado
                
        elif isinstance(d, list):
            for item in d:
                resultado = busca_recursiva(item)
                if resultado is not None:
                    return resultado
        return None

    return busca_recursiva(dicionario)

In [26]:
def get_dictcode(d, code):
    # Se d for um dicionário, iteramos sobre suas chaves e valores
    if isinstance(d, dict):
        if d.get('code') == code:
            return d.get('value')  # Retorna o valor se a chave 'code' for encontrada com o valor especificado
        for value in d.values():
            result = get_dictcode(value, code)  # Chamada recursiva
            if result is not None:
                return result  # Retorna o resultado se o dicionário for encontrado

    # Se d for uma lista, iteramos sobre os elementos da lista
    elif isinstance(d, list):
        for item in d:
            result = get_dictcode(item, code)  # Chamada recursiva
            if result is not None:
                return result  # Retorna o resultado se o dicionário for encontrado

    return None  # Retorna None se o dicionário não for encontrado

In [27]:
def format_desc(x):
    for rep in ['<b>', '</b>', '<br>', '</br>', '<p>', '</p>', '\r', '\n', r'"""¿']:
        x = x.replace(rep, '')
    
    return x

# Constants

In [28]:
CATEGORIAS = [
    'alimentos', 
    'beleza-e-perfumaria',
    'bebidas', 
    'bebidas-alcoolicas-', 
    'limpeza', 
    'bebes-e-criancas', 
    'cuidados-pessoais', 
    'suplementos-alimentares', 
    'eventos-e-festas',
    'utensilios-e-descartaveis',
    'petshop', 
    'esporte-e-lazer',
    'casa-e-construcao',
    'floricultura-e-jardim'
]

# Code

In [30]:
map = []

for category in CATEGORIAS:
    print(f"Obtendo códigos de produtos da categoria {category}...")
    codigos = get_category(category)
    
    print(f"Obtendo informações dos produtos da categoria {category}...")
    # map[category] = fetch_products(codigos)
    for item in fetch_products(codigos, 0):
        map.append({**item, **{'categoria': category}})

map

Obtendo códigos de produtos da categoria alimentos...
Obtendo informações dos produtos da categoria alimentos...
Obtendo códigos de produtos da categoria beleza-e-perfumaria...
Obtendo informações dos produtos da categoria beleza-e-perfumaria...
Obtendo códigos de produtos da categoria bebidas...
Obtendo informações dos produtos da categoria bebidas...
Obtendo códigos de produtos da categoria bebidas-alcoolicas-...
Obtendo informações dos produtos da categoria bebidas-alcoolicas-...
Obtendo códigos de produtos da categoria limpeza...
Obtendo informações dos produtos da categoria limpeza...
Obtendo códigos de produtos da categoria bebes-e-criancas...
Obtendo informações dos produtos da categoria bebes-e-criancas...
Obtendo códigos de produtos da categoria cuidados-pessoais...
Obtendo informações dos produtos da categoria cuidados-pessoais...
Obtendo códigos de produtos da categoria suplementos-alimentares...
Obtendo informações dos produtos da categoria suplementos-alimentares...
Obtend

[{'id': 82810,
  'nome': 'Café em pó 3 Corações Tradicional 500g',
  'sku': '0590655',
  'descricao': '"<p> Há mais de 40 anos, o Café 3 Corações Tradicional leva a mesa dos consumidores, o prazer do sabor equilibrado, forte, com aroma intenso e que promove a cada amanhecer, experiências apaixonantes ao lado de quem mais amamos. \r\nUm clássico com qualidade de origem mineira, trazendo de suas raízes o gosto perfeito a cada xícara. </p>"\r\n',
  'imagem': 'https://static.paodeacucar.com/img/uploads/1/552/28859552.png',
  'preco': 22.99,
  'marca': '3 Corações',
  'alturaProduto': '21',
  'larguraProduto': '11.7',
  'pesoBruto': '513.4',
  'unidadePesoBruto': 'G',
  'profundidadeProduto': '6.8',
  'categoria': 'alimentos'},
 {'id': 273343,
  'nome': 'Massa de Tapioca DA TERRINHA  Pacote 500g',
  'sku': '1021136',
  'descricao': 'MASSA TAPIOCA DA TERRINHA 500G',
  'imagem': 'https://static.paodeacucar.com/img/uploads/1/793/554793.jpg',
  'preco': 8.99,
  'marca': 'Da Terrinha',
  'altura

In [31]:
df = pd.concat([pd.DataFrame([item]) for item in map], ignore_index=True, sort=False, axis='index')
df

,id,nome,sku,descricao,imagem,preco,marca,alturaProduto,larguraProduto,pesoBruto,unidadePesoBruto,profundidadeProduto,categoria
0,82810,Café em pó 3 Corações Tradicional 500g,0590655,"""<p> Há mais de 40 anos, o Café 3 Corações Tra...",https://static.paodeacucar.com/img/uploads/1/5...,22.99,3 Corações,21,11.7,513.4,G,6.8,alimentos
1,273343,Massa de Tapioca DA TERRINHA Pacote 500g,1021136,MASSA TAPIOCA DA TERRINHA 500G,https://static.paodeacucar.com/img/uploads/1/7...,8.99,Da Terrinha,19.6,15,512.1,G,3,alimentos
2,120523,Ovos Branco Extra QUALITÁ Bandeja com 20 Unidades,1510409,"Fonte de nutrientes como proteína, minerais, v...",https://static.paodeacucar.com/img/uploads/1/4...,18.90,Qualitá,7.4,29.1,1,KG,28.4,alimentos
3,655414,Iogurte Parcialmente Desnatado Morango Danone ...,1281652,"O iogurte líquido 1,25KG no sabor morango é pr...",https://static.paodeacucar.com/img/uploads/1/8...,16.90,Danone,24.4,9.9,1.291,KG,9.9,alimentos
4,125224,Brocolis Ninja e Couve Flor QUALITÁ Bandeja 300g,3321195,<b>Ingredientes:</b><br>Brócolis ninja e couve...,https://static.paodeacucar.com/img/uploads/1/2...,11.90,Qualitá,8.9,14.8,316,G,16.1,alimentos
...,...,...,...,...,...,...,...,...,...,...,...,...,...
712,433571,Gel Acendedor FIAT LUX 420g,1182325,,https://static.paodeacucar.com/img/uploads/1/3...,16.99,Fiat Lux,19.9,6.6,483.9,G,6.6,casa-e-construcao
713,1614834,Acendedor Gel para Carvão e Lenha 80° INPM Qua...,1379084,,https://static.paodeacucar.com/img/uploads/1/1...,13.79,Qualitá,21,7.85,455,G,4.9,casa-e-construcao
714,454150,Terra vegetal Premium West Garden 2kg,1216344,,https://static.paodeacucar.com/img/uploads/1/5...,11.99,West Garden,4,16,2,KG,29,floricultura-e-jardim
715,1420972,Vaso Cachepot Siena 16x14cm Rosa Coral,1342365,,https://static.paodeacucar.com/img/uploads/1/5...,49.99,West Garden,14.8,16,100,G,16,floricultura-e-jardim


In [32]:
df['descricao'] = df['descricao'].apply(format_desc)
df['descricao'] = df['descricao'].fillna('Sem descrição')
df['pesoBruto'] = df['pesoBruto'].astype(float)
df['pesoBruto'] = df.apply(lambda x: x['pesoBruto'] if x['unidadePesoBruto'] == 'G' else x['pesoBruto'] * 1000, axis=1)
df.drop(columns=['unidadePesoBruto'], inplace=True)
df

,id,nome,sku,descricao,imagem,preco,marca,alturaProduto,larguraProduto,pesoBruto,profundidadeProduto,categoria
0,82810,Café em pó 3 Corações Tradicional 500g,0590655,""" Há mais de 40 anos, o Café 3 Corações Tradic...",https://static.paodeacucar.com/img/uploads/1/5...,22.99,3 Corações,21,11.7,513.4,6.8,alimentos
1,273343,Massa de Tapioca DA TERRINHA Pacote 500g,1021136,MASSA TAPIOCA DA TERRINHA 500G,https://static.paodeacucar.com/img/uploads/1/7...,8.99,Da Terrinha,19.6,15,512.1,3,alimentos
2,120523,Ovos Branco Extra QUALITÁ Bandeja com 20 Unidades,1510409,"Fonte de nutrientes como proteína, minerais, v...",https://static.paodeacucar.com/img/uploads/1/4...,18.90,Qualitá,7.4,29.1,1000.0,28.4,alimentos
3,655414,Iogurte Parcialmente Desnatado Morango Danone ...,1281652,"O iogurte líquido 1,25KG no sabor morango é pr...",https://static.paodeacucar.com/img/uploads/1/8...,16.90,Danone,24.4,9.9,1291.0,9.9,alimentos
4,125224,Brocolis Ninja e Couve Flor QUALITÁ Bandeja 300g,3321195,Ingredientes:Brócolis ninja e couve flor.,https://static.paodeacucar.com/img/uploads/1/2...,11.90,Qualitá,8.9,14.8,316.0,16.1,alimentos
...,...,...,...,...,...,...,...,...,...,...,...,...
712,433571,Gel Acendedor FIAT LUX 420g,1182325,,https://static.paodeacucar.com/img/uploads/1/3...,16.99,Fiat Lux,19.9,6.6,483.9,6.6,casa-e-construcao
713,1614834,Acendedor Gel para Carvão e Lenha 80° INPM Qua...,1379084,,https://static.paodeacucar.com/img/uploads/1/1...,13.79,Qualitá,21,7.85,455.0,4.9,casa-e-construcao
714,454150,Terra vegetal Premium West Garden 2kg,1216344,,https://static.paodeacucar.com/img/uploads/1/5...,11.99,West Garden,4,16,2000.0,29,floricultura-e-jardim
715,1420972,Vaso Cachepot Siena 16x14cm Rosa Coral,1342365,,https://static.paodeacucar.com/img/uploads/1/5...,49.99,West Garden,14.8,16,100.0,16,floricultura-e-jardim


In [33]:
df.to_csv(r'..\data\produtos.csv', index=False, sep='|', encoding='utf-8')